In [1]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, RegressorMixin
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math 
import itertools

In [2]:
dataset=pd.read_csv("iris-setosa.csv")

X = dataset.columns[1:3]
X = dataset[X]
y = dataset.columns[0]
y = dataset[y]

X = np.array(X)
y = np.array(y)
X=X.astype(float)
y=y.astype(float)
y=np.where(y==0,-1,y)

In [3]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma):
    # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

In [17]:
from cvxopt import matrix
class HYP_SVM(BaseEstimator, RegressorMixin):
    
    def __init__(self, kernel=gaussian_kernel, sigma=None, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
        self.sigma = sigma
        
    #geometric mean
    def score(self, y_predict, y_test):
        print(y_predict)
        test_min=0
        test_max=0
        pred_min=0
        pred_max=0
        y_test=np.asarray(y_test)
        for i in range(0,len(y_test)):
            if(y_test[i]==1):
                test_min=test_min+1
            else:
                test_max=test_max+1
        #print("y_test min",test_min)       
        #print("y_test max",test_max)
        for i in range(0,len(y_predict)):
            if(y_predict[i]==1 and y_predict[i]==y_test[i]):
                pred_min=pred_min+1
            elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
                pred_max=pred_max+1
        #print("y_pred min",pred_min)       
        #print("y_pred max",pred_max)
        se=pred_min/test_min
        sp=pred_max/test_max
        #print(se,sp)
        gm=math.sqrt(se*sp)
        print("GM",gm)    
          
    def get_params(self, deep=True):
        return {"C": self.C, "sigma": self.sigma}        
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def m_func(self, X_train, y):
        n_samples, n_features = X_train.shape 
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        X_train=np.asarray(X_train)
        K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K1[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        #print(K1.shape)
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a_org=a
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        self.sv_yorg=y
        self.kernel = gaussian_kernel
        X_train=np.asarray(X_train)
        b = 0
        for n in range(len(self.a)):
            b += self.sv_y[n]
            b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        b /= len(self.a)
       # print(self.a_org[1])
        #print(self.a_org.shape,self.sv_yorg.shape,K.shape)
        w_phi=0
        total=0
        for n in range(len(self.a_org)):
            w_phi = self.a_org[n] * self.sv_yorg[n] * K1[n] 
        self.d_hyp=np.zeros(n_samples)
        for n in range(len(self.a_org)):
            self.d_hyp += self.sv_yorg[n]*(w_phi+b)
        func=np.zeros((n_samples))
        func=np.asarray(func)
        typ=1
        if(typ==1):
            for i in range(n_samples):
                func[i]=1-(self.d_hyp[i]/(np.amax(self.d_hyp[i])+0.000001))
        beta=0.8
        if(typ==2):
            for i in range(n_samples):
                func[i]=2/(1+beta*self.d_hyp[i])
        r_max=103/4074
        r_min=1
        self.m=func[0:115]*r_min
        #print(self.m.shape)
        self.m=np.append(self.m,func[115:5473]*r_max)
        #print(self.m.shape)
        
 ##############################################################################

    #prendeva come argomento anche x_test, l'ho tolto, ho aggiunto K
    def fit(self, X_train, y):
        
        self.m_func(X_train, y)
        self.kernel = gaussian_kernel
        n_samples, n_features = X_train.shape 

        # Gram matrix

        #print(self.K.shape)

        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        for i in range(n_samples):
            sv=np.logical_or(self.a_org <self.m, self.a_org > 1e-5)
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        #print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        self.b /= len(self.a)
        #print(self.b)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else :
            self.w = None
            
        return self    
        
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv, self.sigma)
                    #print(gaussian_kernel(X[i], sv, self.sigma)>0)
                y_predict[i] = s
                #print(y_predict[i]+self.b)
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [18]:
if __name__ == "__main__":
    import pylab as pl           
    def hyp_svm():
        C_vals = [1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4]
        sigma = [9e-2, 9e-1, 9e+1, 9e+2, 9e+3, 9e+4]
        combs = list(itertools.product(C_vals, sigma))
        
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            typ = 2            
            clf = HYP_SVM(C=100.0, kernel=gaussian_kernel, sigma=0.9)    
    
            #clf.m_func(X_train, y_train)
                
            clf.fit(X_train, y_train)
            y_predict = clf.predict(X_test)

            clf.score(y_predict,y_test)
            #print(y_test)
            correct = np.sum(y_predict == y_test)
            mse = mean_squared_error(y_test, y_predict)
            print("%d out of %d predictions correct" % (correct, len(y_predict)))
            print("Accuracy",correct/len(y_predict))
            print("Errore quadratico medio: ", mse)
            
            
    hyp_svm()


     pcost       dcost       gap    pres   dres
 0:  2.1256e+04 -2.3472e+05  4e+05  1e-01  5e-14
 1:  1.2402e+04 -2.6659e+04  5e+04  1e-02  8e-14
 2:  2.9119e+03 -4.9426e+03  8e+03  1e-03  4e-14
 3:  4.1107e+02 -6.4678e+02  1e+03  1e-14  2e-14
 4:  3.6536e+01 -1.0589e+02  1e+02  7e-16  9e-15
 5: -5.1808e+00 -5.1212e+01  5e+01  2e-14  4e-15
 6: -2.2776e+01 -4.8389e+01  3e+01  2e-14  4e-15
 7: -2.9537e+01 -3.5336e+01  6e+00  2e-15  5e-15
 8: -3.1239e+01 -3.2869e+01  2e+00  1e-14  4e-15
 9: -3.1895e+01 -3.2042e+01  1e-01  3e-14  5e-15
10: -3.1960e+01 -3.1961e+01  2e-03  1e-14  4e-15
11: -3.1960e+01 -3.1961e+01  2e-05  1e-14  5e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.1256e+04 -2.3472e+05  4e+05  1e-01  5e-14
 1:  1.2402e+04 -2.6659e+04  5e+04  1e-02  8e-14
 2:  2.9119e+03 -4.9426e+03  8e+03  1e-03  4e-14
 3:  4.1107e+02 -6.4678e+02  1e+03  1e-14  2e-14
 4:  3.6536e+01 -1.0589e+02  1e+02  7e-16  9e-15
 5: -5.1808e+00 -5.1212e+01  5e+01  2e-14  4e-1

In [6]:
#clf_svm = svm.SVC(kernel='rbf', gamma=0.001, C=100)
#clf_svm.fit(X_train, y_train)
#y_pred_svm = clf_svm.predict(X_test) 
#acc_svm = accuracy_score(y_test, y_pred_svm)
#print ("Overall RBF KERNEL SVM accuracy: ",acc_svm)

In [15]:
C_vals = [1e-2, 1e-1, 1, 1e+1, 1e+2, 1e+3, 1e+4]
sigma = [9e-2, 9e-1, 9, 9e+1, 9e+2, 9e+3, 9e+4]
#kernels = ["linear_kernel", "polynomial_kernel", "gaussian_kernel"]
parameters = {'C': C_vals, 'sigma': sigma}

model = HYP_SVM(C=100, kernel=gaussian_kernel, sigma=0.9)

xTrain, xTest, yTrain, yTest = train_test_split(X, y)

clf = GridSearchCV(model, parameters, cv=5)

In [19]:
grid_result = clf.fit(X=xTrain, y=yTrain)

#clf.score(xTest, yTest)
#clf.best_params_


     pcost       dcost       gap    pres   dres
 0: -1.5496e+01 -6.3071e+00  4e+02  2e+01  3e-16
 1: -5.3696e+00 -2.1583e+00  3e+01  2e+00  4e-16
 2: -6.4709e-01 -1.6454e+00  1e+00  2e-02  1e-15
 3: -5.9140e-01 -6.5369e-01  6e-02  2e-17  7e-16
 4: -5.9486e-01 -5.9979e-01  5e-03  2e-17  3e-16
 5: -5.9547e-01 -5.9608e-01  6e-04  2e-17  3e-16
 6: -5.9558e-01 -5.9565e-01  7e-05  2e-17  4e-16
 7: -5.9559e-01 -5.9560e-01  7e-06  2e-17  4e-16
 8: -5.9559e-01 -5.9559e-01  3e-07  2e-17  3e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.5496e+01 -6.3071e+00  4e+02  2e+01  3e-16
 1: -5.3696e+00 -2.1583e+00  3e+01  2e+00  4e-16
 2: -6.4709e-01 -1.6454e+00  1e+00  2e-02  1e-15
 3: -5.9140e-01 -6.5369e-01  6e-02  2e-17  7e-16
 4: -5.9486e-01 -5.9979e-01  5e-03  2e-17  3e-16
 5: -5.9547e-01 -5.9608e-01  6e-04  2e-17  3e-16
 6: -5.9558e-01 -5.9565e-01  7e-05  2e-17  4e-16
 7: -5.9559e-01 -5.9560e-01  7e-06  2e-17  4e-16
 8: -5.9559e-01 -5.9559e-01  3e-07  2e-17  3e-1

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
from cvxopt import matrix
class SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def fit(self, X, y):
        self.kernel = gaussian_kernel
        n_samples, n_features = X.shape
        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = gaussian_kernel(X[i], X[j])
               # print(K[i,j])
        print(K.shape)

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
       # print(a)
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
                #print(self.w)
        else:
            self.w = None

    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [ ]:
if __name__ == "__main__":
    import pylab as pl           
    def normal_svm():
        
        clf = SVM(C=100.0)
        
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        for train_index, test_index in skf.split(X, y):
            #print("TRAIN:", train_index, "\nTEST:", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            clf.fit(X_train, y_train)    
            y_predict = clf.predict(X_test)
            gm(y_predict,y_test)
            correct = np.sum(y_predict == y_test)
            mse = mean_squared_error(y_test, y_predict)
            print("%d out of %d predictions correct" % (correct, len(y_predict)))
            print("Accuracy",correct/len(y_predict))
            print("Errore quadratico medio: ", mse)
    normal_svm()   